# Структуры для представления триангуляции

## Мотивация

Выбор структуры для представления *триангуляции* оказывает существенное влияние на теоретическую трудоёмкость алгоритмов, а также на скорость конкретной реализации. Кроме того, выбор структуры может зависеть от цели дальнейшего использования триангуляции.

## Краткий ликбез

*Определение 1.* **Триангуляцией** называется планарный граф, все внутренние области которого являются треугольниками.

*Определение 2.* **Выпуклой триангуляцией** называется такая *триангуляция*, для которой минимальный многоугольник, охватывающий все треугольники, будет выпуклым.

*Определение 3.* Задачей **построения триангуляции** по заданному набору двумерных точек называется задача соединения заданных точек непересекающимися отрезками так, чтобы образовалась *триангуляция*. Заданный набор точек будем называть *точками триангуляции*.

*Определение 4.* *Триангуляция* называется **оптимальной**, если сумма длин всех рёбер минимальна среди всех возможных триангуляций, построенных на тех же *точках триангуляции*.

Задача построения оптимальной триангуляции является достаточно трудоёмкой. На практике применяют приближенные алгоритмы, например, широко известна *триангуляция Делоне*.

*Определение 5.* Говорят, что *триангуляция* удовлетворяет **условию Делоне**, если внутрь окружности, описанной вокруг любого построенного треугольника, не попадает ни одна из *точек триангуляции*.

*Определение 6.* *Триангуляция* называется **триангуляцией Делоне**, если она является выпуклой и удовлетворяет *условию Делоне*.

## Основные виды объектов триангуляции, операции с ними

В триангуляции выделяют 3 основных вида объектов: **узлы** (точки, вершины), **рёбра** (отрезки) и **треугольники**.

В большинстве алгоритмов построения триангуляции Делоне и алгоритмов её анализа требуются следующие операции с объектами триангуляции:

1. Треугольник → узлы: получение для данного треугольника координат образующих его узлов.
2. Треугольник → рёбра: получение для данного треугольника списка образующих его рёбер.
3. Треугольник → треугольники: получение для данного треугольника списка соседних с ним треугольников.
4. Ребро → узлы: получение для данного ребра координат образующих его узлов.
5. Ребро → треугольники: получение для данного ребра списка соседних с ним треугольников.
6. Узел → рёбра: получение для данного узла списка смежных рёбер.
7. Узел → треугольники: получение для данного узла списка смежных треугольников.

В зависимости от алгоритма некоторые операции могут быть не нужны. Например, в каких-то алгоритмах операции с рёбрами могут возникать не часто, поэтому рёбра могут представляться неявно, как одна из сторон некоторого треугольника. 

## Наиболее часто используемые структуры

### "Узлы с соседями"

В структуре «Узлы с соседями» для каждого узла триангуляции хранятся его координаты на плоскости и список указателей на соседние узлы (список номеров узлов), с которыми есть общие рёбра. По сути, список соседей определяет в неявном виде рёбра триангуляции. Треугольники же при этом не представляются вообще. 

In [1]:
from typing import List

class NodeWithNeighbours(object):
    """
    Структура "Узел с соседями"
    
    :param x:     координата X
    :param y:     координата Y
    :param count: количество смежных узлов Делоне 
    :param nodes: список смежных узлов
    """
    def __init__(x : float, y : float, count : int, nodes : List['NodeWithNeighbours']):
        self.x = x
        self.y = y
        self.count = count
        self.nodes = nodes

**Замечания**: отстуствие представления треугольников, а так же недостатком является переменный размер структуры узла, приводящий к неэкономному расходу оперативной памяти при построении триангуляции. 

**Память**: учитывая, что среднее число смежных узлов в триангуляции Делоне равно 6, то при 8-байтовом представлении координат, 4-байтовых целых (count) и 4-байтовых указателях суммарный объем памяти, занимаемый данной структурой триангуляцией, составляет **44\*N байт**, где N - количество узлов.

### "Двойные рёбра"

В структуре «Двойные рёбра» основой триангуляции является список ориентированных рёбер. При этом каждое ребро входит в структуру триангуляцию дважды, но направленными в противоположные стороны.

Для каждого ребра хранятся следующие указатели:
1. на концевой узел ребра
2. на следующее по часовой стрелке ребро в треугольнике, находящемся справа от данного ребра
3. на «ребро-близнец», соединяющее те же самые узлы триангуляции, что и данное, но направленное в противоположную сторону 
4. на треугольник, находящийся справа от ребра

Последний указатель не нужен для построения триангуляции, и поэтому его наличие должно определяться в зависимости от цели дальнейшего применения триангуляции. 

In [2]:
class DoubleEdges(object):
    """
    Структура "Двойные ребра"
    Представляет собой классы Node, Edge и Triangle
    
    """
    
    class Node(object):
        """
        Класс Узел
         
        :param x: координата X
        :param y: координата Y
        """
        def __init__(x : float, y : float):
            self.x = x
            self.y = y
    
    class Edge(object):
        """
        Класс Ребро
        
        :param node:     концевой узел ребра
        :param nxt:      следующее ребро по часовой стрелке в треугольнике справа
        :param twin:     ребро-близнец, направленное в другую сторону
        :param triangle: треугольник справа
        """
        def __init__(node : 'Node', nxt : 'Edge', twin : 'Edge', triangle : 'Triangle'):
            self.node = node
            self.nxt = nxt
            self.twin = twin
            self.triangle = triangle
        
    class Triangle(object):
        """
        Класс Треугольник
        
        Поля определяются в зависимости от поставленных задач
        """
        pass 

![alt text](https://pp.userapi.com/c840531/v840531372/15226/cyTSzYe8wSg.jpg "Двойные рёбра")
Рисунок 1. Связи рёбер и неявное задание треугольников в структуре «Двойные рёбра» 

**Замечания**: представление треугольников в неявном виде, а также большой расход памяти.

**Память**: при 8-байтовом представлении координат и 4-байтовых указателях получается **не менее 64\*N байт** (+ расход памяти на представление дополнительных
данных в треугольниках).

### "Узлы и треугольники"

В структуре «Узлы и треугольники» для каждого треугольника хранятся три указателя на образующие его узлы и три указателя на смежные треугольники.

Нумерация точек и соседних треугольников производится в порядке обхода по часовой стрелке, при этом напротив точки с номером i = 1..3 располагается ребро, соответствующее соседнему треугольнику с таким же номером.
Рёбра в данной триангуляции в явном виде не хранятся. При необходимости же они обычно представляются как указатель на треугольник и номер ребра внутри него. 

In [3]:
class NodesWithTriangles(object):
    """
    Структура "Узлы и треугольники"
    Представляет собой классы Node и Triangle
    
    """
    
    class Node(object):
        """
        Класс Узел
         
        :param x: координата X
        :param y: координата Y
        """
        def __init__(x : float, y : float):
            self.x = x
            self.y = y
        
    class Triangle(object):
        """
        Класс Треугольник
         
        :param nodes:     образующие треугольник узлы
        :param triangles: соседние треугольники
        """
        def __init__(nodes : List['Node'], triangles : List['Triangle']):
            self.nodes = nodes
            self.triangles = triangles

**Замечания**: несмотря на то, что данная структура уступает «Узлам с соседями» по памяти, она наиболее часто применяется на практике в силу своей относительной простоты и удобства программирования алгоритмов на её основе. 

**Память:** при 8-байтовом представлении координат и 4-байтовых указателях данная структура триангуляции требует примерно **64\*N** байт.

![alt text](https://pp.userapi.com/c841233/v841233812/2deed/cW0jRET0il0.jpg "Узлы и треугольники")
Рисунок 2. Связи узлов и треугольников в структуре «Узлы и треугольники» 

### "Узлы, рёбра и треугольники"

В структуре «Узлы, рёбра и треугольники» в явном виде задаются все объекты триангуляции: узлы, рёбра и треугольники. Для каждого ребра хранятся указатели на два концевых узла и два соседних треугольника. Для треугольников хранятся указатели на три образующих треугольник ребра. 

In [4]:
class NodesAndEdgesAndTriangles(object):
    """
    Структура "Узлы, ребра и треугольники"
    
    Представляет собой классы Node, Edge и Triangle
    """
    
    class Node(object):
        """
        Класс Узел
         
        :param x: координата X
        :param y: координата Y
        """
        def __init__(x : float, y : float):
            self.x = x
            self.y = y
            
    class Edge(object):
        """
        Класс Ребро
         
        :param nodes:     список концевых узлов
        :param triangles: соседние треугольники
        """
        def __init__(nodes : List['Node'], triangles : List['Triangle']):
            self.nodes = nodes
            self.triangles = triangles
            
    class Triangle(object):
        """
        Класс Треугольник
        
        :param edges: образующие ребра
        """
        def __init__(edges : List['Edge']):
            self.edges = edges

**Замечания**: большой расход памяти, хотя данная структура часто применяется на практике, особенно в задачах, где требуется в явном виде представлять рёбра триангуляции.

**Память**: при 8-байтовом представлении координат и 4-байтовых указателях примерно **88\*N байт**. 

### "Узлы, простые рёбра и треугольники"

В структуре «Узлы, простые рёбра и треугольники» в явном виде задаются все объекты триангуляции: узлы, рёбра и треугольники. Для каждого ребра хранятся указатели на два концевых узла и два соседних треугольника. Для рёбер никакой специальной информации нет. Для треугольников хранятся указатели на образующих треугольник три узла и три
ребра, а также указатели на три смежных треугольника. 

In [5]:
class NodesAndSimpleEdgesAndTriangles(object):
    """
    Структура "Узлы, простые ребра и треугольники"
    
    Представляет собой классы Node, SimpleEdge и Triangle
    """
    
    class Node(object):
        """
        Класс Узел
         
        :param x: координата X
        :param y: координата Y
        """
        def __init__(x : float, y : float):
            self.x = x
            self.y = y
            
    class Edge(object):
        """
        Класс Ребро
         
        Поля определяются в зависимости от поставленных задач
        """
        pass
            
    class Triangle(object):
        """
        Класс Треугольник
        
        :param nodes:     образующие узлы
        :param triangles: соседние треугольники
        :param edges:     образующе ребра
        """
        def __init__(nodes : List['Node'], triangles : List['Triangle'], edges : List['Edge']):
            self.nodes = nodes
            self.triangles = triangles
            self.edges = edges

**Замечания:** данная структура часто применяется на практике, особенно в задачах, где требуется в явном виде представлять рёбра триангуляции. Недостатком данной структуры является относительно большой расход памяти.

**Память**: при 8-байтовом представлении координат и 4-байтовых указателях примерно **80\*N байт**. 

## Резюме и практические рекомендации

Таблица 1. Основные характеристики структур: «–» – элемент отсутствует, «+» – присутствует, «+-» – присутствует, но нет связей с другими элементами триангуляции.

| Название структуры данных                | Память               | Узлы      | Рёбра     | Треугольники     |
|------------------------------------------|:--------------------:|----------:|-----------|------------------|
| "Узлы с соседями"                        | **44\*N**            | **+**     |  **-**    |     **-**        |
| "Двойные рёбра"                          | **64\*N**            | **+-**    |  **+**    |     **+-**       |
| "Узлы и треугольники"                    | **64\*N**            | **+-**    |  **-**    |     **+**        |
| "Узлы, рёбра и треугольники"             | **88\*N**            | **+-**    |  **+**    |     **+**        |
| "Узлы, простые рёбра и треугольники"     | **80\*N**            | **+-**    |  **+-**   |     **+**        |


Выводы относительно рассмотренных структур:
* "Узлы с соседями" наименее удобная и используемая структура, так как не представляет в явном виде рёбра и треугольники.

* Удобной с точки зрения программирования является структура "Узлы и треугольники".

* Для алгоритмов, требующих представление рёбер в явном виде, рекомендуется структура "Узлы, рёбра и треугольники".

### Примеры алгоритмов, в которых эффективно использовать структуры

Итеративный алгоритм триангуляции Делоне (без ограничений):
* "Узлы и треугольники"

Цепной алгоритм построения триангуляции с ограничениями (требуется представлять рёбра в явном виде):
* "Двойные рёбра"
* "Узлы, рёбра и треугольники"

Триангуляция Делоне с ограничениями:
* "Узлы, простые рёбра и треугольники" (рёбра в явном виде, так как для рёбер необходимо хранить дополнительную информацию о том, являются ли они структурными, а так же простой переход к структуре от итеративного алгоритма триангуляции Делоне без ограничений) 



